In [ ]:
# OpenStreetMAp Example frok lesson 8

#!/usr/bin/env python
# -*- coding: utf-8 -*-
import xml.etree.cElementTree as ET
from collections import defaultdict
import re

osm_file = open("chicago.osm", "r")

street_type_re = re.compile(r'\S+\.?$', re.IGNORECASE)
street_types = defaultdict(int)

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()

        street_types[street_type] += 1

def print_sorted_dict(d):
    keys = d.keys()
    keys = sorted(keys, key=lambda s: s.lower())
    for k in keys:
        v = d[k]
        print "%s: %d" % (k, v) 

def is_street_name(elem):
    return (elem.tag == "tag") and (elem.attrib['k'] == "addr:street")

def audit():
    for event, elem in ET.iterparse(osm_file):
        if is_street_name(elem):
            audit_street_type(street_types, elem.attrib['v'])    
    print_sorted_dict(street_types)    

if __name__ == '__main__':
    audit()

In [ ]:
"""
Your task is to check the "productionStartYear" of the DBPedia autos datafile for valid values.
The following things should be done:
- check if the field "productionStartYear" contains a year
- check if the year is in range 1886-2014
- convert the value of the field to be just a year (not full datetime)
- the rest of the fields and values should stay the same
- if the value of the field is a valid year in the range as described above,
  write that line to the output_good file
- if the value of the field is not a valid year as described above, 
  write that line to the output_bad file
- discard rows (neither write to good nor bad) if the URI is not from dbpedia.org
- you should use the provided way of reading and writing data (DictReader and DictWriter)
  They will take care of dealing with the header.

You can write helper functions for checking the data and writing the files, but we will call only the 
'process_file' with 3 arguments (inputfile, output_good, output_bad).
"""

# This is my solution to the problem

import csv
import pprint

INPUT_FILE = 'autos.csv'
OUTPUT_GOOD = 'autos-valid.csv'
OUTPUT_BAD = 'FIXME-autos.csv'

def process_file(input_file, output_good, output_bad):
    f_good = open(output_good, "w")
    f_bad = open(output_bad, "w")
    

    with open(input_file, "r") as f:
        reader = csv.DictReader(f)
        header = reader.fieldnames

        #COMPLETE THIS FUNCTION
        writer_good = csv.DictWriter(f_good, delimiter=",", fieldnames= header)
        writer_good.writeheader()
        writer_bad = csv.DictWriter(f_bad, delimiter=",", fieldnames= header)
        writer_bad.writeheader()
        #print header, "\n"
        for row in reader:
            if (row["URI"].strip() != "URI") and (row["URI"].startswith("http://dbpedia.org")):
            #if row["URI"].startswith("http://dbpedia.org"):
                #http://dbpedia.org/resource/Crawler-transporter
                #print type(row["productionStartYear"]), row["productionStartYear"]
                if row["productionStartYear"] != "NULL":
                    if int(row["productionStartYear"][0:1] == "{"):
                        year = int(row["productionStartYear"][1:5])
                    else:
                        year = int(row["productionStartYear"][0:4])
                    if (year >= 1886) and (year <= 2014):
                        row["productionStartYear"] = year
                        writer_good.writerow(row)
                        print year
                    else:
                        writer_bad.writerow(row)
                        #print row
                else:
                    writer_bad.writerow(row)
                    #print row


    '''
    # This is just an example on how you can use csv.DictWriter
    # Remember that you have to output 2 files
    with open(output_good, "w") as g:
        writer = csv.DictWriter(g, delimiter=",", fieldnames= header)
        writer.writeheader()
        for row in YOURDATA:
            writer.writerow(row)
    '''
    

def test():

    process_file(INPUT_FILE, OUTPUT_GOOD, OUTPUT_BAD)


if __name__ == "__main__":
    test()

In [ ]:
# This is thw instructor solution for same problem above

def process_file(input_file, output_good, output_bad):
    # store data into lists for output
    data_good = []
    data_bad = []
    with open(input_file, "r") as f:
        reader = csv.DictReader(f)
        header = reader.fieldnames
        for row in reader:
            # validate URI value
            if row['URI'].find("dbpedia.org") < 0:
                continue

            ps_year = row['productionStartYear'][:4]
            try: # use try/except to filter valid items
                ps_year = int(ps_year)
                row['productionStartYear'] = ps_year
                if (ps_year >= 1886) and (ps_year <= 2014):
                    data_good.append(row)
                else:
                    data_bad.append(row)
            except ValueError: # non-numeric strings caught by exception
                if ps_year == 'NULL':
                    data_bad.append(row)

    # Write processed data to output files
    with open(output_good, "w") as good:
        writer = csv.DictWriter(good, delimiter=",", fieldnames= header)
        writer.writeheader()
        for row in data_good:
            writer.writerow(row)

    with open(output_bad, "w") as bad:
        writer = csv.DictWriter(bad, delimiter=",", fieldnames= header)
        writer.writeheader()
        for row in data_bad:
            writer.writerow(row)

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
In this problem set you work with cities infobox data, audit it, come up with a
cleaning idea and then clean it up. In the first exercise we want you to audit
the datatypes that can be found in some particular fields in the dataset.
The possible types of values can be:
- NoneType if the value is a string "NULL" or an empty string ""
- list, if the value starts with "{"
- int, if the value can be cast to int
- float, if the value can be cast to float, but CANNOT be cast to int.
   For example, '3.23e+07' should be considered a float because it can be cast
   as float but int('3.23e+07') will throw a ValueError
- 'str', for all other values

The audit_file function should return a dictionary containing fieldnames and a 
SET of the types that can be found in the field. e.g.
{"field1": set([type(float()), type(int()), type(str())]),
 "field2": set([type(str())]),
  ....
}
The type() function returns a type object describing the argument given to the 
function. You can also use examples of objects to create type objects, e.g.
type(1.1) for a float: see the test function below for examples.

Note that the first three rows (after the header row) in the cities.csv file
are not actual data points. The contents of these rows should note be included
when processing data types. Be sure to include functionality in your code to
skip over or detect these rows.
"""

# this code did not pass the test bottoon but passed when submitted!!!!!
# wasted tooooo much gtime on this....not sure why!!!!

import codecs
import csv
import json
import pprint

CITIES = 'cities.csv'

FIELDS = ["name", "timeZone_label", "utcOffset", "homepage", "governmentType_label",
          "isPartOf_label", "areaCode", "populationTotal", "elevation",
          "maximumElevation", "minimumElevation", "populationDensity",
          "wgs84_pos#lat", "wgs84_pos#long", "areaLand", "areaMetro", "areaUrban"]

def audit_file(filename, fields):
    fieldtypes = {}

    # YOUR CODE HERE
    #type_list = set()
    str = ""
    with open(filename, "r") as f:
        reader = csv.DictReader(f)
        header = reader.fieldnames
        type_list = []

        
        test_fields = ["areaLand", "areaMetro"]
        #for field in fields:
        for field in test_fields:
        #print field
            for row in reader:

                # validate URI value
                if row['URI'].find("dbpedia.org") < 0:
                    continue

                field_value = row[field]
                if field_value != 'NULL':
                    if field_value[0:1] == "{":
                        if type(type_list) not in type_list:
                            type_list.append(type(type_list))
                            #print field_value
                    else:
                        try: # use try/except to filter valid items
                            number = int(field_value)
                            if type(number) not in type_list:
                                type_list.append(type(number))
                                #print "An integer"

                        except ValueError: # use try/except to filter valid items
                            if field_value.count('.') > 0:
                                number = float(field_value)
                                if type(number) not in type_list:
                                    type_list.append(type(number))
                                    #print "A float"
                            else:
                                if type(str) not in type_list:
                                    type_list.append(type(""))
                                    #print field_value

                            #print field_value
                else:
                    if type(None) not in type_list:
                        type_list.append(type(None))
                        #print field_value
            
            fieldtypes[field] = set(type_list)
            print fieldtypes[field]
            #del type_list[:]
            type_list = []
            
            #print fieldtypes
            #print fieldtypes["areaLand"]
            #print fieldtypes["areaMetro"]
            
    return fieldtypes


def test():
    fieldtypes = audit_file(CITIES, FIELDS)

    pprint.pprint(fieldtypes)
    #print set([type(1.1), type(None)])

    assert fieldtypes["areaLand"] == set([type(1.1), type([]), type(None)])
    assert fieldtypes['areaMetro'] == set([type(1.1), type(None)])
    
if __name__ == "__main__":
    test()


In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
In this problem set you work with cities infobox data, audit it, come up with a
cleaning idea and then clean it up.

Since in the previous quiz you made a decision on which value to keep for the
"areaLand" field, you now know what has to be done.

Finish the function fix_area(). It will receive a string as an input, and it
has to return a float representing the value of the area or None.
You have to change the function fix_area. You can use extra functions if you
like, but changes to process_file will not be taken into account.
The rest of the code is just an example on how this function can be used.
"""
import codecs
import csv
import json
import pprint

CITIES = 'cities.csv'

# function to return the number of significant digits in a string representation of a number either in
# scientific or decimal notation
# refer to https://www.chem.tamu.edu/class/fyp/mathrev/mr-sigfg.html for definition used for significant digits
def significant_digits(numStr):
    
    if ('e' in numStr):     # if string is using sientific notation
        temp_str = numStr.split('e')            # remove the e and return the len of number to left of e
        num_of_sig_digits = len( temp_str[0] ) - 1  # length minus the decimal point 
        #print "scientific notation in significant_digits function ", numStr, num_of_sig_digits
    else:
        if '.' in numStr:   # if string is in decimal notation
            temp_str = numStr.replace('.', '')  # just remove the decimal point
            num_of_sig_digits = len(temp_str)
            #print "decimal notation in significant_digits function ", numStr, num_of_sig_digits

        else:       # string is in non-decimal or integer notation
            num_of_sig_digits = len(numStr) - len(numStr.rstrip('0'))   # remove trailing 0's as in 300 -> 1
            #print "integer notation in significant_digits function ", numStr, num_of_sig_digits
    
    return num_of_sig_digits
    
def fix_area(area):

    # YOUR CODE HERE
    
    sig_digits_list = []
    digits_list = []
    if area != "NULL":
        if area[0:1] != "{":
            if ('e' in area):
                #s = area.split('e')
                #sig_digits = len( s[0] ) - 1
                #sig_digits = significant_digits(area)
                #print "scientific notation ", area, sig_digits
                area = float(area)
            else:
                #print "real number  ", area
                area = float(area)
                
        else:       # parse the list of number to and return the number with most significany digits
            s = area.split('|')
            for num in s:
                num = num.strip('{')
                num = num.strip('}')
                #print "list -----", num
                digits_list.append(num)         # keep the stripeed numbers in a list
                sig_digits_list.append(significant_digits(num))     # keep the number of significant difgits in a list
            i = sig_digits_list.index(max(sig_digits_list))     # find the index of the number with higher sig digits
            area = float(digits_list[i])        # retueb the float of the number with higher sig digits
            #print  "figure with more significant digits ", max(sig_digits_list), area

    else:
        area = None
    

    return area



def process_file(filename):
    # CHANGES TO THIS FUNCTION WILL BE IGNORED WHEN YOU SUBMIT THE EXERCISE
    data = []

    with open(filename, "r") as f:
        reader = csv.DictReader(f)

        #skipping the extra metadata
        for i in range(3):
            l = reader.next()

        # processing file
        for line in reader:
            # calling your function to fix the area value
            if "areaLand" in line:
                line["areaLand"] = fix_area(line["areaLand"])
            data.append(line)

    return data


def test():
    data = process_file(CITIES)

    print "Printing three example results:"
    for n in range(5,8):
        pprint.pprint(data[n]["areaLand"])

    assert data[3]["areaLand"] == None        
    assert data[8]["areaLand"] == 55166700.0
    assert data[20]["areaLand"] == 14581600.0
    assert data[33]["areaLand"] == 20564500.0    


if __name__ == "__main__":
    test()

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
In this problem set you work with cities infobox data, audit it, come up with a
cleaning idea and then clean it up.

In the previous quiz you recognized that the "name" value can be an array (or
list in Python terms). It would make it easier to process and query the data
later if all values for the name are in a Python list, instead of being
just a string separated with special characters, like now.

Finish the function fix_name(). It will recieve a string as an input, and it
will return a list of all the names. If there is only one name, the list will
have only one item in it; if the name is "NULL", the list should be empty.
The rest of the code is just an example on how this function can be used.
"""
import codecs
import csv
import pprint

CITIES = 'cities.csv'


def fix_name(name):

    # YOUR CODE HERE
    names_list = []
    
    if name != "NULL":
        if name[0:1] == "{":
                
            names = name.split('|')
            for n in names:
                n = n.strip('{')
                n = n.strip('}')
                names_list.append(n)
                #print ">>>>>>>>>>>>>" ,n
        else:
            names_list.append(name)
    else:   # just return the empty list
        pass

    #return name
    #print len(names_list), names_list
    return names_list


def process_file(filename):
    data = []
    with open(filename, "r") as f:
        reader = csv.DictReader(f)
        #skipping the extra metadata
        for i in range(3):
            l = reader.next()
        # processing file
        for line in reader:
            # calling your function to fix the area value
            if "name" in line:
                line["name"] = fix_name(line["name"])
            data.append(line)
    return data


def test():
    data = process_file(CITIES)

    print "Printing 20 results:"
    for n in range(20):
        pprint.pprint(data[n]["name"])

    assert data[14]["name"] == ['Negtemiut', 'Nightmute']
    assert data[9]["name"] == ['Pell City Alabama']
    assert data[3]["name"] == ['Kumhari']

if __name__ == "__main__":
    test()

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
In this problem set you work with cities infobox data, audit it, come up with a
cleaning idea and then clean it up.

If you look at the full city data, you will notice that there are couple of
values that seem to provide the same information in different formats: "point"
seems to be the combination of "wgs84_pos#lat" and "wgs84_pos#long". However,
we do not know if that is the case and should check if they are equivalent.

Finish the function check_loc(). It will recieve 3 strings: first, the combined
value of "point" followed by the separate "wgs84_pos#" values. You have to
extract the lat and long values from the "point" argument and compare them to
the "wgs84_pos# values, returning True or False.

Note that you do not have to fix the values, only determine if they are
consistent. To fix them in this case you would need more information. Feel free
to discuss possible strategies for fixing this on the discussion forum.

The rest of the code is just an example on how this function can be used.
Changes to "process_file" function will not be taken into account for grading.
"""
import csv
import pprint

CITIES = 'cities.csv'


def check_loc(point, lat, longi):
    # YOUR CODE HERE
    
    lattitude, longtitude = point.split()
    print point, lattitude, longtitude
    return (lattitude == lat) and (longtitude == longi)
    

def process_file(filename):
    data = []
    with open(filename, "r") as f:
        reader = csv.DictReader(f)
        #skipping the extra matadata
        for i in range(3):
            l = reader.next()
        # processing file
        for line in reader:
            # calling your function to check the location
            result = check_loc(line["point"], line["wgs84_pos#lat"], line["wgs84_pos#long"])
            if not result:
                print "{}: {} != {} {}".format(line["name"], line["point"], line["wgs84_pos#lat"], line["wgs84_pos#long"])
            data.append(line)

    return data


def test():
    assert check_loc("33.08 75.28", "33.08", "75.28") == True
    assert check_loc("44.57833333333333 -91.21833333333333", "44.5783", "-91.2183") == False

if __name__ == "__main__":
    test()